In [3]:
import numpy as np;
import pandas as pd;
import csv, sqlite3
from pyhive import presto
import pymysql;

inact_drivers_rfm_table = pd.read_csv("bi_inactive_driver_rfm_data.csv")
print(len(inact_drivers_rfm_table))
inact_drivers_rfm_table.head()

52858


,id_driver,recency,frequency,contribution,updated_at
0,693724,239,9,9,2020-03-22 07:16:47.834 UTC
1,86496,177,52,411,2020-03-22 07:16:47.834 UTC
2,602751,284,8,8,2020-03-22 07:16:47.834 UTC
3,427012,361,55,320,2020-03-22 07:16:47.834 UTC
4,674333,94,21,236,2020-03-22 07:16:47.834 UTC


In [4]:
quantiles = inact_drivers_rfm_table.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()
inact_drivers_rfm_table['r_quartile'] = 0

def RScore(x,d):
    if x[d] <= 60:
        return 1
    elif x[d] <= 90:
        return 2
    elif x[d] <= 120: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

inact_drivers_rfm_table.head()

,id_driver,recency,frequency,contribution,updated_at,r_quartile
0,693724,239,9,9,2020-03-22 07:16:47.834 UTC,0
1,86496,177,52,411,2020-03-22 07:16:47.834 UTC,0
2,602751,284,8,8,2020-03-22 07:16:47.834 UTC,0
3,427012,361,55,320,2020-03-22 07:16:47.834 UTC,0
4,674333,94,21,236,2020-03-22 07:16:47.834 UTC,0


In [5]:
segmented_rfm = inact_drivers_rfm_table
inact_drivers_rfm_table['r_quartile'].loc[(inact_drivers_rfm_table['recency'] <= 60) & (inact_drivers_rfm_table['recency'] >= 30)] = 1
inact_drivers_rfm_table['r_quartile'].loc[(inact_drivers_rfm_table['recency'] <= 90) & (inact_drivers_rfm_table['recency'] > 60)] = 2
inact_drivers_rfm_table['r_quartile'].loc[(inact_drivers_rfm_table['recency'] <= 120) & (inact_drivers_rfm_table['recency'] > 90)] = 3
inact_drivers_rfm_table['r_quartile'].loc[(inact_drivers_rfm_table['recency'] >120)] = 4

segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['contribution'].apply(FMScore, args=('contribution',quantiles,))
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
print(segmented_rfm.head())
segmented_rfm.groupby('f_quartile')['frequency'].describe()
segmented_rfm.groupby('r_quartile')['recency'].describe()

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


   id_driver  recency  frequency  contribution                   updated_at  \
0     693724      239          9             9  2020-03-22 07:16:47.834 UTC   
1      86496      177         52           411  2020-03-22 07:16:47.834 UTC   
2     602751      284          8             8  2020-03-22 07:16:47.834 UTC   
3     427012      361         55           320  2020-03-22 07:16:47.834 UTC   
4     674333       94         21           236  2020-03-22 07:16:47.834 UTC   

   r_quartile  f_quartile  m_quartile RFMScore  
0           4           3           4      434  
1           4           2           2      422  
2           4           3           4      434  
3           4           2           2      422  
4           3           3           2      332  


,count,mean,std,min,25%,50%,75%,max
r_quartile,,,,,,,,
1,8815.0,44.613046,8.633718,31.0,37.0,44.0,52.0,60.0
2,7232.0,75.193722,8.295633,61.0,68.0,75.0,82.0,90.0
3,5117.0,105.696111,8.883932,91.0,98.0,106.0,114.0,120.0
4,31694.0,230.884962,71.443290,121.0,166.0,225.0,290.0,365.0


In [6]:
seg = {'111' : 'POWER',
'112' : 'CORE',
'211' : 'CORE',
'212' : 'CORE',
'312' : 'CORE',
'311' : 'CORE',
'113' : 'PROMISE',
'314' : 'PROMISE',
'313' : 'PROMISE',
'114' : 'PROMISE',
'123' : 'PROMISE',
'324' : 'PROMISE',
'224' : 'PROMISE',
'124' : 'PROMISE',
'223' : 'PROMISE',
'424' : 'PROMISE',
'413' : 'PROMISE',
'414' : 'PROMISE',
'214' : 'PROMISE',
'213' : 'PROMISE',
'323' : 'PROMISE',
'332' : 'LOYAL',
'342' : 'LOYAL',
'321' : 'LOYAL',
'331' : 'LOYAL',
'141' : 'LOYAL',
'241' : 'LOYAL',
'421' : 'LOYAL',
'131' : 'LOYAL',
'231' : 'LOYAL',
'221' : 'LOYAL',
'441' : 'LOYAL',
'121' : 'LOYAL',
'431' : 'LOYAL',
'132' : 'LOYAL',
'222' : 'LOYAL',
'122' : 'LOYAL',
'232' : 'LOYAL',
'242' : 'LOYAL',
'142' : 'LOYAL',
'411' : 'LOYAL',
'341' : 'LOYAL',
'322' : 'LOYAL',
'412' : 'IDLE',
'442' : 'IDLE',
'423' : 'IDLE',
'433' : 'IDLE',
'422' : 'IDLE',
'432' : 'IDLE',
'434' : 'IDLE',
'444' : 'IDLE',
'443' : 'IDLE',
'343' : 'LOW',
'144' : 'LOW',
'344' : 'LOW',
'133' : 'LOW',
'244' : 'LOW',
'334' : 'LOW',
'134' : 'LOW',
'234' : 'LOW',
'243' : 'LOW',
'143' : 'LOW',
'233' : 'LOW',
'333' : 'LOW'}

def rfm_switch(x):
    z= seg[x]
    return z

segmented_rfm['RFM_segment'] = segmented_rfm['RFMScore'].apply(rfm_switch)

In [7]:
segmented_rfm['segment_type']="inactive"
seg_test=segmented_rfm[['id_driver','RFM_segment','RFMScore','r_quartile','f_quartile','m_quartile','recency','frequency','contribution','segment_type','updated_at']]
seg_test.head()

,id_driver,RFM_segment,RFMScore,r_quartile,f_quartile,m_quartile,recency,frequency,contribution,segment_type,updated_at
0,693724,IDLE,434,4,3,4,239,9,9,inactive,2020-03-22 07:16:47.834 UTC
1,86496,IDLE,422,4,2,2,177,52,411,inactive,2020-03-22 07:16:47.834 UTC
2,602751,IDLE,434,4,3,4,284,8,8,inactive,2020-03-22 07:16:47.834 UTC
3,427012,IDLE,422,4,2,2,361,55,320,inactive,2020-03-22 07:16:47.834 UTC
4,674333,LOYAL,332,3,3,2,94,21,236,inactive,2020-03-22 07:16:47.834 UTC


In [30]:
## Inactive Passengers

inact_passengers_rfm_table = pd.read_csv("bi_inactive_passenger_rfm_data.csv")
print(len(inact_passengers_rfm_table))
inact_passengers_rfm_table.head()

132310


,id_passenger,recency,frequency,monetary_value,updated_at
0,4042975,50,2,28,2020-03-22 07:17:05.745 UTC
1,4043222,89,2,16,2020-03-22 07:17:05.745 UTC
2,4044856,205,1,1,2020-03-22 07:17:05.745 UTC
3,2802065,281,1,20,2020-03-22 07:17:05.745 UTC
4,2809448,74,1,93,2020-03-22 07:17:05.745 UTC


In [31]:
len(inact_passengers_rfm_table)

132310

In [32]:
quantiles = inact_passengers_rfm_table.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

#quantiles_r = inact_passengers_rfm_table.quantile(q=[0.33,0.66])
#quantiles_r = quantiles_r.to_dict()

In [33]:
inact_passengers_rfm_table['r_quartile'] = 0

In [34]:
#fixed Recency figures
#Quartile Frequency &  

def RScore(x,d):
    if x[d] <= 60:
        return 1
    elif x[d] <= 90:
        return 2
    elif x[d] <= 120: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [35]:
inact_passengers_rfm_table.head()

,id_passenger,recency,frequency,monetary_value,updated_at,r_quartile
0,4042975,50,2,28,2020-03-22 07:17:05.745 UTC,0
1,4043222,89,2,16,2020-03-22 07:17:05.745 UTC,0
2,4044856,205,1,1,2020-03-22 07:17:05.745 UTC,0
3,2802065,281,1,20,2020-03-22 07:17:05.745 UTC,0
4,2809448,74,1,93,2020-03-22 07:17:05.745 UTC,0


In [36]:
segmented_rfm = inact_passengers_rfm_table

In [37]:
inact_passengers_rfm_table['r_quartile'].loc[(inact_passengers_rfm_table['recency'] <= 60) & (inact_passengers_rfm_table['recency'] >= 30)] = 1
inact_passengers_rfm_table['r_quartile'].loc[(inact_passengers_rfm_table['recency'] <= 90) & (inact_passengers_rfm_table['recency'] > 60)] = 2
inact_passengers_rfm_table['r_quartile'].loc[(inact_passengers_rfm_table['recency'] <= 120) & (inact_passengers_rfm_table['recency'] > 90)] = 3
inact_passengers_rfm_table['r_quartile'].loc[(inact_passengers_rfm_table['recency'] >120)] = 4

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [38]:
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()

,id_passenger,recency,frequency,monetary_value,updated_at,r_quartile,f_quartile,m_quartile
0,4042975,50,2,28,2020-03-22 07:17:05.745 UTC,1,2,2
1,4043222,89,2,16,2020-03-22 07:17:05.745 UTC,2,2,2
2,4044856,205,1,1,2020-03-22 07:17:05.745 UTC,4,4,4
3,2802065,281,1,20,2020-03-22 07:17:05.745 UTC,4,4,2
4,2809448,74,1,93,2020-03-22 07:17:05.745 UTC,2,4,1


In [39]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,id_passenger,recency,frequency,monetary_value,updated_at,r_quartile,f_quartile,m_quartile,RFMScore
0,4042975,50,2,28,2020-03-22 07:17:05.745 UTC,1,2,2,122
1,4043222,89,2,16,2020-03-22 07:17:05.745 UTC,2,2,2,222
2,4044856,205,1,1,2020-03-22 07:17:05.745 UTC,4,4,4,444
3,2802065,281,1,20,2020-03-22 07:17:05.745 UTC,4,4,2,442
4,2809448,74,1,93,2020-03-22 07:17:05.745 UTC,2,4,1,241


In [40]:
segmented_rfm.groupby('f_quartile')['frequency'].describe()

,count,mean,std,min,25%,50%,75%,max
f_quartile,,,,,,,,
1,16408.0,4.599037,2.791412,3.0,3.0,4.0,5.0,48.0
2,23588.0,2.000000,0.000000,2.0,2.0,2.0,2.0,2.0
4,92314.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0


In [41]:
segmented_rfm.groupby('r_quartile')['recency'].describe()

,count,mean,std,min,25%,50%,75%,max
r_quartile,,,,,,,,
1,32462.0,43.544390,8.451369,31.0,36.0,43.0,50.0,60.0
2,17715.0,75.225064,8.762590,61.0,67.0,75.0,83.0,90.0
3,15453.0,104.847538,8.892863,91.0,97.0,105.0,113.0,120.0
4,66680.0,231.761233,70.885742,121.0,169.0,227.0,292.0,365.0


In [42]:
seg = {'111' : 'DORMANT HIGH',
'112' : 'DORMANT HIGH',
'113' : 'DORMANT HIGH',
'114' : 'DORMANT HIGH',
'123' : 'DORMANT HIGH',
'124' : 'DORMANT HIGH',
'121' : 'DORMANT HIGH',
'122' : 'DORMANT HIGH',
'141' : 'DORMANT LOW',
'131' : 'DORMANT LOW',
'132' : 'DORMANT LOW',
'142' : 'DORMANT LOW',
'144' : 'DORMANT LOW',
'133' : 'DORMANT LOW',
'134' : 'DORMANT LOW',
'143' : 'DORMANT LOW',
'211' : 'CHURNED HIGH',
'314' : 'CHURNED HIGH',
'313' : 'CHURNED HIGH',
'212' : 'CHURNED HIGH',
'214' : 'CHURNED HIGH',
'213' : 'CHURNED HIGH',
'312' : 'CHURNED HIGH',
'324' : 'CHURNED HIGH',
'224' : 'CHURNED HIGH',
'223' : 'CHURNED HIGH',
'221' : 'CHURNED HIGH',
'222' : 'CHURNED HIGH',
'323' : 'CHURNED HIGH',
'321' : 'CHURNED HIGH',
'311' : 'CHURNED HIGH',
'322' : 'CHURNED HIGH',
'241' : 'CHURNED LOW',
'231' : 'CHURNED LOW',
'232' : 'CHURNED LOW',
'242' : 'CHURNED LOW',
'333' : 'CHURNED LOW',
'341' : 'CHURNED LOW',
'332' : 'CHURNED LOW',
'342' : 'CHURNED LOW',
'331' : 'CHURNED LOW',
'343' : 'CHURNED LOW',
'344' : 'CHURNED LOW',
'244' : 'CHURNED LOW',
'334' : 'CHURNED LOW',
'234' : 'CHURNED LOW',
'243' : 'CHURNED LOW',
'233' : 'CHURNED LOW',
'421' : 'LOST HIGH',
'441' : 'LOST HIGH',
'431' : 'LOST HIGH',
'411' : 'LOST HIGH',
'412' : 'LOST HIGH',
'422' : 'LOST HIGH',
'442' : 'LOST HIGH',
'432' : 'LOST HIGH',
'424' : 'LOST LOW',
'423' : 'LOST LOW',
'433' : 'LOST LOW',
'443' : 'LOST LOW',
'434' : 'LOST LOW',
'444' : 'LOST LOW',
'413' : 'LOST LOW',
'414' : 'LOST LOW'
}

def rfm_switch(x):
    z= seg[x]
    return z

segmented_rfm['RFM_segment'] = segmented_rfm['RFMScore'].apply(rfm_switch)

In [44]:
segmented_rfm['segment_type']="inactive"
seg_test=segmented_rfm[['id_passenger','RFM_segment','RFMScore','r_quartile','f_quartile','m_quartile','recency','frequency','monetary_value','segment_type','updated_at']]
seg_test.head()

,id_passenger,RFM_segment,RFMScore,r_quartile,f_quartile,m_quartile,recency,frequency,monetary_value,segment_type,updated_at
0,4042975,DORMANT HIGH,122,1,2,2,50,2,28,inactive,2020-03-22 07:17:05.745 UTC
1,4043222,CHURNED HIGH,222,2,2,2,89,2,16,inactive,2020-03-22 07:17:05.745 UTC
2,4044856,LOST LOW,444,4,4,4,205,1,1,inactive,2020-03-22 07:17:05.745 UTC
3,2802065,LOST HIGH,442,4,4,2,281,1,20,inactive,2020-03-22 07:17:05.745 UTC
4,2809448,CHURNED LOW,241,2,4,1,74,1,93,inactive,2020-03-22 07:17:05.745 UTC


In [45]:
segmented_rfm.groupby('RFMScore').count()

,id_passenger,recency,frequency,monetary_value,updated_at,r_quartile,f_quartile,m_quartile,RFM_segment,segment_type
RFMScore,,,,,,,,,,
111,1951,1951,1951,1951,1951,1951,1951,1951,1951,1951
112,1172,1172,1172,1172,1172,1172,1172,1172,1172,1172
113,757,757,757,757,757,757,757,757,757,757
121,2602,2602,2602,2602,2602,2602,2602,2602,2602,2602
122,1932,1932,1932,1932,1932,1932,1932,1932,1932,1932
123,1362,1362,1362,1362,1362,1362,1362,1362,1362,1362
124,644,644,644,644,644,644,644,644,644,644
141,6540,6540,6540,6540,6540,6540,6540,6540,6540,6540
142,6024,6024,6024,6024,6024,6024,6024,6024,6024,6024


In [46]:
segmented_rfm.groupby('RFMScore').median()

,id_passenger,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
RFMScore,,,,,,,
111,1534209.0,42.0,3.0,89.0,1.0,1.0,1.0
112,3574132.0,43.0,3.0,17.0,1.0,1.0,2.0
113,4585731.0,43.0,3.0,5.0,1.0,1.0,3.0
121,1384278.5,41.0,2.0,80.0,1.0,2.0,1.0
122,3151603.0,42.0,2.0,17.0,1.0,2.0,2.0
123,4091938.0,43.0,2.0,5.0,1.0,2.0,3.0
124,4672909.5,43.0,2.0,2.0,1.0,2.0,4.0
141,1342429.5,42.0,1.0,73.0,1.0,4.0,1.0
142,2864453.5,43.0,1.0,16.0,1.0,4.0,2.0
